In [ ]:
# import libraries

import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

In [ ]:
# run the data_processing file to get the training and validation sets

%run ../data_processing.ipynb

In [ ]:
# create a baseline classifier
def create_baseline():

    # init a sequential NN
    classifier = Sequential()

    # Define the model architecture
    
    # 1st layer - takes in input
    classifier.add(Dense(units=512, kernel_initializer='he_uniform',
                         activation='relu', input_dim=12))
    # 2nd layer
    classifier.add(Dense(units=128, kernel_initializer='he_uniform',
                         activation='relu'))
    # 3rd layer
    classifier.add(Dense(units=96, kernel_initializer='he_uniform',
                         activation='relu'))
    # output layer
    classifier.add(Dense(units=2, kernel_initializer='he_uniform',
                         activation='softmax'))

    return classifier

In [ ]:
# compile parameters (can be updated for different variations and test)
X_TRAIN = x_train
Y_TRAIN = y_train

X_VALID = x_valid
Y_VALID = y_valid

VALIDATION = (x_valid, y_valid)

EPOCHS = 50
BATCH_SIZE = 32
VERBOSE = 0

# list of learning rates to test (can be updated to test differentlearning rates)
learning_rates = [0.001, 0.003, 0.01, 0.03, 0.1, 0.3]

# dataframe to hold the loss and accuracy for each learning rate
df_result = pd.DataFrame(columns=['learning_rate', 'training_loss',
                                  'training_accuracy', 'validation_loss',
                                  'validation_accuracy'])

# clears the dataset to avoid redundancy
df_result = df_result.iloc[0:0]

In [ ]:
# loop through the learning_rates list to test all the learning rates
for LEARNING_RATE in learning_rates:
    
    # create a baseline model
    model = create_baseline()

    # compile the model with values defined above
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=LEARNING_RATE),
                  loss='categorical_crossentropy', metrics=['accuracy'])
    
    # train the model
    history = model.fit(X_TRAIN, Y_TRAIN, validation_data=VALIDATION,
                            epochs=EPOCHS, batch_size=BATCH_SIZE,
                            verbose=VERBOSE)

    # plot
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid = True
    plt.show()

    # save figure
    plt.savefig('figures/learning_rate_'+str(LEARNING_RATE)+'.png')

    # evaluation - loss and accuracy
    model_train_loss, model_train_accuracy = model.evaluate(X_TRAIN, Y_TRAIN,
                                                            verbose=VERBOSE)
    model_valid_loss, model_valid_accuracy = model.evaluate(X_VALID, Y_VALID,
                                                            verbose=VERBOSE)

    # add the evaluation results to the dataset
    df_result = pd.concat([pd.DataFrame([[LEARNING_RATE, model_train_loss,
                                         model_train_accuracy,
                                         model_valid_loss,
                                         model_valid_accuracy]],
                                         columns=df_result.columns),
                                         df_result],
                                         ignore_index=True)

In [ ]:
# display the evaluation results dataframe

df_result

In [ ]:
# visualize loss for training and validation sets

# x values
x = ['training_loss', 'validation_loss']

# training and validation loss for each learning rate
for _idx in range(df_result.shape[0]):
    plt.plot(x, [df_result.iloc[_idx]['training_loss'],
                 df_result.iloc[_idx]['validation_loss']],
                 label=str(df_result.iloc[_idx]['learning_rate']))

# plot
plt.grid = True
plt.title('Loss')
plt.legend(loc='upper right')
plt.show()

# save figure
plt.savefig('figures/loss.png')

In [ ]:
# visualize accuracy for training and validation sets

# x values
x = ['training_accuracy', 'validation_accuracy']

# training and validation accuracy for each learning rate
for _idx in range(df_result.shape[0]):
    plt.plot(x, [df_result.iloc[_idx]['training_accuracy'],
                 df_result.iloc[_idx]['validation_accuracy']],
                 label=str(df_result.iloc[_idx]['learning_rate']))

# plot
plt.grid = True
plt.title('Accuracy')
plt.legend(loc='upper right')
plt.show()

# save figure
plt.savefig('figures/accuracy.png')